In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from os import sys
from selenium import webdriver
import re
from datetime import date, timedelta
import time

In [169]:
station_code_dic = dict()
station_code_dic["TaiPei"] = "RCSS"
station_code_dic["ShenZhen"] = "ZGSZ"
station_code_dic["Teheran"] = "OIII"
station_code_dic["HangZhou"] = "ZSHC"
station_code_dic["BeiJing"] = "ZBAA"

In [170]:
table = soup.find_all('div', class_='observation-table')
print(len(table))

1


In [181]:
def get_max_temp(city_name : str, _date : date, station_code_dic : dict = station_code_dic) -> int:

    code = station_code_dic[city_name]

    url = f"https://www.wunderground.com/history/daily/{code}/date/{_date.year}-{_date.month}-{_date.day}"
    # print(url)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('div', class_='observation-table')
    
    vals = []
    if table:
        rows = table[0].find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= 2:
                temp = cols[1].find('span', class_="wu-value wu-value-to")
                if temp:
                    raw = str(temp)
                    match = re.search(r'class="wu-value wu-value-to">(\d+)</span>', raw)
                    if match:
                        value = int(match.group(1))
                        vals.append(value)
                        # print(f"Row {n}: {value}°F")

    return max(vals)
    

In [ ]:
def fill_csv_with_weather_data(station_code_dic : dict = station_code_dic) -> None:
    for city in station_code_dic.keys():
        start_date = date(2005,1,1)
        end_date = date(2025,4,15)
        file_name = f"{city}_weather_{start_date}_to_{end_date}.csv"
        
        current_date = start_date

        with open(file_name, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Max Temperature"])

        nDays = 0
        while current_date <= end_date:
            nDays += 1
            try:
                max_temp = get_max_temp(city, current_date)
                max_temp = int(round((max_temp - 32) * 5 / 9, 1)) # Convert to Celcius
            except Exception as e:
                print(f"Error on {city} - {current_date}: {e}")
                max_temp = "-"

            with open(file_name, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([current_date, max_temp])
            
                current_date += timedelta(days=1)
            
            if nDays == 5:
                break
                

In [ ]:
start = time.time()
fill_csv_with_weather_data()
end = time.time()
elapsed = end - start
more_iterations = int(365 / 5 * 20)
estimation_seconds = int(elapsed * more_iterations)
estimation_hours = int(estimation_seconds / 3600)

print(f"The whole process will take approximately {estimation_hours} time")

KeyboardInterrupt: 